In [19]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import lower
import pandas as pd

In [2]:
spark = (SparkSession
 .builder
 .appName('project2_part1')
 .enableHiveSupport()
 .getOrCreate())

23/09/11 19:33:08 WARN Utils: Your hostname, MacBook-Air-Ekaterina-2.local resolves to a loopback address: 127.0.0.1; using 172.20.10.5 instead (on interface en0)
23/09/11 19:33:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/11 19:33:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

# Ex1

In [4]:
data = pd.DataFrame([(1, "athletics", "summer"),
        (2, "swimming", "summer"),
        (3, "cycling", "summer"),
        (4, "gymnastics", "summer"),
        (5, "rowing", "summer"),
        (6, "skiing", "winter"),
        (7, "curling", "winter"),
        (8, "hockey", "winter"),
        (9, "figure_skating", "winter"),
        (10, "snowboarding", "winter")])
data.columns = ['row_id', 'Discipline', 'Season']
data

,row_id,Discipline,Season
0,1,athletics,summer
1,2,swimming,summer
2,3,cycling,summer
3,4,gymnastics,summer
4,5,rowing,summer
5,6,skiing,winter
6,7,curling,winter
7,8,hockey,winter
8,9,figure_skating,winter
9,10,snowboarding,winter


In [5]:
spark_data = spark.createDataFrame(data)

In [6]:
spark_data.show()

+------+--------------+------+
|row_id|    Discipline|Season|
+------+--------------+------+
|     1|     athletics|summer|
|     2|      swimming|summer|
|     3|       cycling|summer|
|     4|    gymnastics|summer|
|     5|        rowing|summer|
|     6|        skiing|winter|
|     7|       curling|winter|
|     8|        hockey|winter|
|     9|figure_skating|winter|
|    10|  snowboarding|winter|
+------+--------------+------+



In [7]:
spark_data.coalesce(1).write.csv('olympic_disciplines.csv', header = True, sep = '\t')

In [8]:
ls

Athletes.csv             README.md
Code_part1.ipynb         olympic_disciplines.csv/


# Ex2

In [9]:
athletes = spark.read.csv('Athletes.csv', header = True, sep = ';')

In [23]:
athletes = athletes.withColumn("Discipline", lower("Discipline"))

In [24]:
athletes.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       cycling road|
|         ABAD Nestor|               Spain|artistic gymnastics|
|   ABAGNALE Giovanni|               Italy|             rowing|
|      ABALDE Alberto|               Spain|         basketball|
|       ABALDE Tamara|               Spain|         basketball|
|           ABALO Luc|              France|           handball|
|        ABAROA Cesar|               Chile|             rowing|
|       ABASS Abobakr|               Sudan|           swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             karate|
|       ABBASOV Islam|          Azerbaijan|          wrestling|
|        ABBINGH Lois|         Netherlands|           handball|
|         ABBOT Emily|           Australia|rhythmic gymnastics|
|       ABBOTT Monica|United States of .

In [25]:
df2 =athletes.groupBy("Discipline").count().withColumnRenamed("count","Total athletes")
df2.show()

+--------------------+--------------+
|          Discipline|Total athletes|
+--------------------+--------------+
|              hockey|           406|
|    beach volleyball|            90|
|            shooting|           342|
|        cycling road|           190|
|      sport climbing|            37|
|          water polo|           269|
|   marathon swimming|            49|
|          equestrian|           237|
|      3x3 basketball|            62|
|          basketball|           280|
|             sailing|           336|
|trampoline gymnas...|            31|
|  cycling bmx racing|            43|
|   artistic swimming|            98|
|           triathlon|           106|
|             archery|           122|
|             surfing|            38|
|        table tennis|           164|
|           wrestling|           279|
|        canoe sprint|           236|
+--------------------+--------------+
only showing top 20 rows



In [26]:
df2.write.parquet('total_athletes.parquet')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/ekaterinaprokopeva/DE_Project_Part_2/total_athletes.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

# Ex3

3. Прочитайте исходный файл "Athletes.csv".
Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.
Получившийся результат нужно объединить с сгенерированным вами DataFrame из 1-го задания и в итоге вывести количество участников, только по тем дисциплинам, что есть в вашем сгенерированном DataFrame.
Результат сохраните в формате parquet.


In [27]:
athletes.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       cycling road|
|         ABAD Nestor|               Spain|artistic gymnastics|
|   ABAGNALE Giovanni|               Italy|             rowing|
|      ABALDE Alberto|               Spain|         basketball|
|       ABALDE Tamara|               Spain|         basketball|
|           ABALO Luc|              France|           handball|
|        ABAROA Cesar|               Chile|             rowing|
|       ABASS Abobakr|               Sudan|           swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             karate|
|       ABBASOV Islam|          Azerbaijan|          wrestling|
|        ABBINGH Lois|         Netherlands|           handball|
|         ABBOT Emily|           Australia|rhythmic gymnastics|
|       ABBOTT Monica|United States of .

In [35]:
merged_df = athletes.join(spark_data, 'Discipline', 'inner').groupBy('Discipline').count()

In [36]:
merged_df.show()

+----------+-----+
|Discipline|count|
+----------+-----+
| athletics| 2068|
|  swimming|  743|
|    rowing|  496|
|    hockey|  406|
+----------+-----+



In [37]:
merged_df.write.parquet('filtered_athletes.parquet')